In [23]:
import os

In [24]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops'

In [11]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops'

In [5]:
# Entity

In [68]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str 
    alpha: float 
    l1_ratio: float 
    target_column: str 

In [69]:
# Configuration Manager

In [70]:
from CarPrediction.constants import *
from CarPrediction.utils.common import read_yaml, create_directories

In [71]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer 
          params = self.params.ElasticNet 
          schema = self.schema.TARGET_COLUMN
          
          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir=config.root_dir,
                train_data_path=config.train_data_path,
                test_data_path=config.test_data_path,
                model_name= config.model_name,
                alpha = params.alpha,
                l1_ratio=params.l1_ratio,
                target_column= schema.name
          )

          return model_trainer_config

In [72]:
# Components

In [73]:
import pandas as pd
import os 
from CarPrediction import logger 
from sklearn.linear_model import ElasticNet 
import joblib 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [74]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target variable
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Define a column transformer for preprocessing
        numeric_features = train_x.select_dtypes(include=['int32']).columns
        numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

        categorical_features = train_x.select_dtypes(include=['object']).columns
        categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # Combine preprocessing with the model
        model = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=101))])

        # Fit the model
        model.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [75]:
# Pipeline

In [76]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-09 14:46:02,608: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-09 14:46:02,611: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-09 14:46:02,615: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-09 14:46:02,618: INFO: common: created directory at: artifacts]
[2024-01-09 14:46:02,619: INFO: common: created directory at: artifacts/model_trainer]
